# IBM Data Science Capstone
### This Notebook will be mainly for the Capstone Project

#### Week 1

In [1]:
import pandas as pd
import numpy as np
import requests
import folium
from IPython.display import display
import matplotlib.cm as cm
import matplotlib.colors as colors

In [2]:
import folium
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim

In [3]:
print("Hello Capstone Project Course")

Hello Capstone Project Course


#### Week 2

In [4]:
url  = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = requests.get(url)

In [5]:
df = pd.read_html(url, header=0, na_values = ['Not assigned'])[0]
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,NaN,NaN
1,M2A,NaN,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [6]:
#Drop Boroughs with Empty set
df.dropna(subset=['Borough'], inplace=True)

In [7]:
n_empty_neighborhood = df[df['Neighborhood'].isna()].shape[0]

if n_empty_neighborhood == 0:
    print("There are no rows with an empty value")
else: 
    print(df[df['Neighbourhood'].isna()])

There are no rows with an empty value


In [8]:
df_postcodes = df.groupby(['Postal Code','Borough']).Neighborhood.agg([('Neighborhood', ', '.join)])
df_postcodes.reset_index(inplace=True)
df_postcodes.head(5)

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [9]:
num_rows, num_column = df_postcodes.shape
print('The Number of Rows in the Dataframe is', num_rows)

The Number of Rows in the Dataframe is 103


In [10]:
#Export to .CSV
df_postcodes.to_csv('Toronto_Postcodes.csv')

In [11]:
#Reading CSV file
url_csv = 'http://cocl.us/Geospatial_data'
df_coordinates = pd.read_csv(url_csv)
df_coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [12]:
df_neighborhoods = pd.read_csv('Toronto_Postcodes.csv',index_col=[0])
df_neighborhoods.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [13]:
#use the previously cleaned data
df_neighborhoods = pd.read_csv('Toronto_Postcodes.csv',index_col=[0])
df_neighborhoods.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [14]:
# Merge both datasets
df_neighborhoods_coordinates = pd.merge(df_neighborhoods, df_coordinates, on='Postal Code')
df_neighborhoods_coordinates.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [15]:
#counting borough and neighborhoods
df.groupby('Borough').count()['Neighborhood']

Borough
Central Toronto      9
Downtown Toronto    19
East Toronto         5
East York            5
Etobicoke           12
Mississauga          1
North York          24
Scarborough         17
West Toronto         6
York                 5
Name: Neighborhood, dtype: int64

In [16]:
df_toronto = df_neighborhoods_coordinates[df_neighborhoods_coordinates['Borough'].str.contains('Toronto')]
df_toronto.reset_index(inplace=True)
df_toronto.drop('index', axis=1, inplace=True)
df_toronto.head()

C:\Users\olart\anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [17]:
#Count only Toronto
print(df_toronto.groupby('Borough').count()['Neighborhood'])

Borough
Central Toronto      9
Downtown Toronto    19
East Toronto         5
West Toronto         6
Name: Neighborhood, dtype: int64


In [18]:
#Datalist for map
boroughs = df_toronto['Borough'].unique().tolist()
lat = df_toronto['Latitude'].mean()
lon = df_toronto['Longitude'].mean()

In [19]:
map_toronto = folium.Map(location=[lat, lon], zoom_start=5)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighborhood']):
    label_text = borough + ' - ' + neighborhood
    label = folium.Popup(label_text)
    folium.CircleMarker(
        [lat, lng],radius=5,popup=label,color="red",fill_color="red",fill_opacity=0.5).add_to(map_toronto)  

map_toronto

# Target Problem: Is it viable to open a Coffee Shop? If so, where?

#### The data I would be using is the Foursquare API wherein I would determine the viability of opening a coffee shop in terms of present competition. I would recommend then opening a coffee shop in the cluster wherein there are less open coffee shops; therefore, less competition.

In [20]:
#foursquare details
CLIENT_ID = 'CFDXX0Q3JO0MB4SGXDHOSH5M5MKI1GXKR3WUKSLJVMBKJ5NC' 
CLIENT_SECRET = 'XSG4ZM4ZQQS5KXBTNQEYSJM3HUGTOQKC4JD52XREXWFNUGOL' 
VERSION = '20200618' 
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

In [21]:
#function of getting nearby venues
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


In [23]:
#Get venues for all neighborhoods in our dataset
venues = getNearbyVenues(names=df_toronto['Neighborhood'],latitudes=df_toronto['Latitude'], longitudes=df_toronto['Longitude'])

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West, Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West, Forest Hill Road Park
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High 

In [24]:
#checking dataframe
venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West, Riverdale",43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop
...,...,...,...,...,...,...,...
1622,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,Toronto Yoga Mamas,43.664824,-79.324335,Yoga Studio
1623,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,TTC Stop #03049,43.664470,-79.325145,Light Rail Station
1624,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,Greenwood Cigar & Variety,43.664538,-79.325379,Smoke Shop
1625,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,Revolution Recording,43.662561,-79.326940,Recording Studio


In [27]:
#Number of venues per neighborhood
venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,58,58,58,58,58,58
"Brockton, Parkdale Village, Exhibition Place",24,24,24,24,24,24
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",19,19,19,19,19,19
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
Central Bay Street,67,67,67,67,67,67
Christie,17,17,17,17,17,17
Church and Wellesley,80,80,80,80,80,80
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,35,35,35,35,35,35


In [28]:
venues['Venue Category'].unique()[:100]

array(['Trail', 'Health Food Store', 'Pub', 'Neighborhood',
       'Cosmetics Shop', 'Greek Restaurant', 'Ice Cream Shop',
       'Italian Restaurant', 'Brewery', 'Fruit & Vegetable Store',
       'Yoga Studio', 'Pizza Place', 'Restaurant', 'Juice Bar',
       'Bookstore', 'Bubble Tea Shop', 'Dessert Shop',
       'Furniture / Home Store', 'Grocery Store', 'Spa', 'Coffee Shop',
       'Bakery', 'Caribbean Restaurant', 'Indian Restaurant',
       'Japanese Restaurant', 'Café', 'Lounge', 'Frozen Yogurt Shop',
       'Liquor Store', 'American Restaurant', 'Gym', 'Fish & Chips Shop',
       'Fast Food Restaurant', 'Sushi Restaurant', 'Park', 'Pet Store',
       'Steakhouse', 'Burrito Place', 'Movie Theater', 'Sandwich Place',
       'Light Rail Station', 'Intersection', 'Fish Market', 'Gay Bar',
       'Thai Restaurant', 'Seafood Restaurant', 'Cheese Shop',
       'Comfort Food Restaurant', 'Middle Eastern Restaurant',
       'Stationery Store', 'Coworking Space', 'Wine Bar',
       'Latin

In [30]:
# one hot encoding
jap_onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
jap_onehot['Neighborhoods'] = venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [jap_onehot.columns[-1]] + list(jap_onehot.columns[:-1])
jap_onehot = jap_onehot[fixed_columns]

print(jap_onehot.shape)
jap_onehot.head()

(1627, 241)


,Neighborhoods,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"The Danforth West, Riverdale",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
jap = jap_onehot.groupby(["Neighborhoods"]).mean().reset_index()

jap

,Neighborhoods,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Berczy Park,0.0000,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.017241,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.0000,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000
2,"Business reply mail Processing Centre, South C...",0.0000,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.052632
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.0000,0.0625,0.0625,0.0625,0.125,0.1875,0.0625,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000
4,Central Bay Street,0.0000,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.014925,0.000000,0.000000,0.014925,0.0000,0.000000,0.014925
5,Christie,0.0000,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000
6,Church and Wellesley,0.0125,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,0.012500,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0125,0.000000,0.025000
7,"Commerce Court, Victoria Hotel",0.0000,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,0.040000,0.000000,...,0.000000,0.000000,0.00,0.020000,0.000000,0.000000,0.010000,0.0000,0.000000,0.000000
8,Davisville,0.0000,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,0.000000,0.000000,...,0.028571,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000
9,Davisville North,0.0000,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000


In [32]:
#checking the density of japanese restaurants per Neighborhood
restaurants = jap[["Neighborhoods", "Japanese Restaurant"]]
restaurants

,Neighborhoods,Japanese Restaurant
0,Berczy Park,0.017241
1,"Brockton, Parkdale Village, Exhibition Place",0.000000
2,"Business reply mail Processing Centre, South C...",0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000
4,Central Bay Street,0.059701
5,Christie,0.000000
6,Church and Wellesley,0.062500
7,"Commerce Court, Victoria Hotel",0.030000
8,Davisville,0.028571
9,Davisville North,0.000000


In [33]:
# set number of clusters
kclusters = 5

restaurant_clustering = restaurants.drop('Neighborhoods', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(restaurant_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

#labels
restaurant_merge = restaurants.copy()
restaurant_merge["Cluster"] = kmeans.labels_

In [34]:
restaurant_merge.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
restaurant_merge

,Neighborhood,Japanese Restaurant,Cluster
0,Berczy Park,0.017241,4
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0
2,"Business reply mail Processing Centre, South C...",0.000000,0
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0
4,Central Bay Street,0.059701,2
5,Christie,0.000000,0
6,Church and Wellesley,0.062500,2
7,"Commerce Court, Victoria Hotel",0.030000,3
8,Davisville,0.028571,3
9,Davisville North,0.000000,0


In [36]:
#merging and sorting
restaurant_merge = restaurant_merge.join(venues.set_index("Neighborhood"), on="Neighborhood")
restaurant_merge

,Neighborhood,Japanese Restaurant,Cluster,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Berczy Park,0.017241,4,43.644771,-79.373306,LCBO,43.642944,-79.372440,Liquor Store
0,Berczy Park,0.017241,4,43.644771,-79.373306,The Keg Steakhouse + Bar - Esplanade,43.646712,-79.374768,Restaurant
0,Berczy Park,0.017241,4,43.644771,-79.373306,Fresh On Front,43.647815,-79.374453,Vegetarian / Vegan Restaurant
0,Berczy Park,0.017241,4,43.644771,-79.373306,Meridian Hall,43.646292,-79.376022,Concert Hall
0,Berczy Park,0.017241,4,43.644771,-79.373306,Hockey Hall Of Fame (Hockey Hall of Fame),43.646974,-79.377323,Museum
...,...,...,...,...,...,...,...,...,...
38,"University of Toronto, Harbord",0.057143,2,43.662696,-79.400049,DT Bistro,43.662375,-79.405734,Café
38,"University of Toronto, Harbord",0.057143,2,43.662696,-79.400049,Comfort Zone,43.658397,-79.400274,Nightclub
38,"University of Toronto, Harbord",0.057143,2,43.662696,-79.400049,The Beer Store,43.665385,-79.403477,Beer Store
38,"University of Toronto, Harbord",0.057143,2,43.662696,-79.400049,Cafe On Harbord,43.662903,-79.403564,Café


In [37]:
#sorting
restaurant_merge.sort_values(["Cluster"], inplace=True)
restaurant_merge

,Neighborhood,Japanese Restaurant,Cluster,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
24,"Regent Park, Harbourfront",0.000000,0,43.654260,-79.360636,Corktown Common,43.655618,-79.356211,Park
21,"North Toronto West, Lawrence Park",0.000000,0,43.715383,-79.405678,A&W,43.715149,-79.399944,Fast Food Restaurant
21,"North Toronto West, Lawrence Park",0.000000,0,43.715383,-79.405678,Sporting Life,43.716277,-79.400248,Sporting Goods Shop
21,"North Toronto West, Lawrence Park",0.000000,0,43.715383,-79.405678,Tim Hortons,43.714894,-79.399776,Coffee Shop
21,"North Toronto West, Lawrence Park",0.000000,0,43.715383,-79.405678,GAP,43.715450,-79.400089,Clothing Store
...,...,...,...,...,...,...,...,...,...
25,"Richmond, Adelaide, King",0.010753,4,43.650571,-79.384568,Pi Co.,43.648651,-79.385874,Pizza Place
25,"Richmond, Adelaide, King",0.010753,4,43.650571,-79.384568,deKEFIR,43.650433,-79.379393,Ice Cream Shop
25,"Richmond, Adelaide, King",0.010753,4,43.650571,-79.384568,Equinox Bay Street,43.648100,-79.379989,Gym
25,"Richmond, Adelaide, King",0.010753,4,43.650571,-79.384568,Kojin,43.649398,-79.386091,Colombian Restaurant


In [38]:
#cluster 0
restaurant_merge.loc[restaurant_merge['Cluster'] == 0]

,Neighborhood,Japanese Restaurant,Cluster,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
24,"Regent Park, Harbourfront",0.0,0,43.654260,-79.360636,Corktown Common,43.655618,-79.356211,Park
21,"North Toronto West, Lawrence Park",0.0,0,43.715383,-79.405678,A&W,43.715149,-79.399944,Fast Food Restaurant
21,"North Toronto West, Lawrence Park",0.0,0,43.715383,-79.405678,Sporting Life,43.716277,-79.400248,Sporting Goods Shop
21,"North Toronto West, Lawrence Park",0.0,0,43.715383,-79.405678,Tim Hortons,43.714894,-79.399776,Coffee Shop
21,"North Toronto West, Lawrence Park",0.0,0,43.715383,-79.405678,GAP,43.715450,-79.400089,Clothing Store
...,...,...,...,...,...,...,...,...,...
28,"Runnymede, Swansea",0.0,0,43.651571,-79.484450,Book City (Bloor West),43.650211,-79.481220,Bookstore
28,"Runnymede, Swansea",0.0,0,43.651571,-79.484450,Fat Bastard Burrito,43.649779,-79.482894,Burrito Place
28,"Runnymede, Swansea",0.0,0,43.651571,-79.484450,Campo,43.655191,-79.487067,Italian Restaurant
28,"Runnymede, Swansea",0.0,0,43.651571,-79.484450,DAVIDsTEA,43.650380,-79.478819,Tea Room


In [39]:
#cluster 1
restaurant_merge.loc[restaurant_merge['Cluster'] == 1]

,Neighborhood,Japanese Restaurant,Cluster,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
36,"The Danforth West, Riverdale",0.023256,1,43.679557,-79.352188,Athen's Pastries,43.678166,-79.348927,Greek Restaurant
30,"St. James Town, Cabbagetown",0.022222,1,43.667967,-79.367675,Ambal Trading,43.667538,-79.369350,Market
30,"St. James Town, Cabbagetown",0.022222,1,43.667967,-79.367675,Tim Hortons / Esso,43.667130,-79.370664,Coffee Shop
36,"The Danforth West, Riverdale",0.023256,1,43.679557,-79.352188,Caffé Demetre,43.677683,-79.351608,Dessert Shop
36,"The Danforth West, Riverdale",0.023256,1,43.679557,-79.352188,IQ Living,43.678477,-79.347811,Furniture / Home Store
...,...,...,...,...,...,...,...,...,...
19,"Little Portugal, Trinity",0.022727,1,43.647927,-79.419750,Union Restaurant,43.645860,-79.419532,New American Restaurant
19,"Little Portugal, Trinity",0.022727,1,43.647927,-79.419750,Gift Shop,43.646149,-79.419481,Cocktail Bar
19,"Little Portugal, Trinity",0.022727,1,43.647927,-79.419750,Paris Paris Bar,43.649237,-79.421436,Wine Bar
36,"The Danforth West, Riverdale",0.023256,1,43.679557,-79.352188,Dolce Gelato,43.677773,-79.351187,Ice Cream Shop


In [40]:
#cluster 2
restaurant_merge.loc[restaurant_merge['Cluster'] == 2]

,Neighborhood,Japanese Restaurant,Cluster,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
38,"University of Toronto, Harbord",0.057143,2,43.662696,-79.400049,Cafe On Harbord,43.662903,-79.403564,Café
38,"University of Toronto, Harbord",0.057143,2,43.662696,-79.400049,East of Brunswick,43.665609,-79.403324,Pub
4,Central Bay Street,0.059701,2,43.657952,-79.387383,Barberian's Steak House,43.657755,-79.382177,Steakhouse
6,Church and Wellesley,0.062500,2,43.665860,-79.383160,Out On The Street,43.668293,-79.380340,Men's Store
6,Church and Wellesley,0.062500,2,43.665860,-79.383160,Wow! Sushi,43.668514,-79.386686,Sushi Restaurant
...,...,...,...,...,...,...,...,...,...
6,Church and Wellesley,0.062500,2,43.665860,-79.383160,Rolltation,43.669388,-79.386566,Sushi Restaurant
6,Church and Wellesley,0.062500,2,43.665860,-79.383160,H Mart,43.669332,-79.386257,Grocery Store
6,Church and Wellesley,0.062500,2,43.665860,-79.383160,Constantine,43.668773,-79.385287,Mediterranean Restaurant
6,Church and Wellesley,0.062500,2,43.665860,-79.383160,Ritz Caribbean Food,43.669545,-79.386506,Caribbean Restaurant


In [41]:
#cluster 3
restaurant_merge.loc[restaurant_merge['Cluster'] == 3]

,Neighborhood,Japanese Restaurant,Cluster,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
7,"Commerce Court, Victoria Hotel",0.03,3,43.648198,-79.379817,Kupfert & Kim (First Canadian Place),43.648547,-79.381624,Gluten-free Restaurant
7,"Commerce Court, Victoria Hotel",0.03,3,43.648198,-79.379817,Petit Four Bakery,43.647744,-79.379588,Sandwich Place
7,"Commerce Court, Victoria Hotel",0.03,3,43.648198,-79.379817,Olly Fresco's,43.646912,-79.379597,Deli / Bodega
7,"Commerce Court, Victoria Hotel",0.03,3,43.648198,-79.379817,John & Sons Oyster House,43.650656,-79.381613,Seafood Restaurant
7,"Commerce Court, Victoria Hotel",0.03,3,43.648198,-79.379817,Hy's Steakhouse,43.649505,-79.382919,Steakhouse
...,...,...,...,...,...,...,...,...,...
11,"First Canadian Place, Underground city",0.03,3,43.648429,-79.382280,John & Sons Oyster House,43.650602,-79.381555,Seafood Restaurant
11,"First Canadian Place, Underground city",0.03,3,43.648429,-79.382280,Rosalinda,43.650252,-79.385156,Vegetarian / Vegan Restaurant
11,"First Canadian Place, Underground city",0.03,3,43.648429,-79.382280,Ki Modern Japanese + Bar,43.647167,-79.379608,Japanese Restaurant
11,"First Canadian Place, Underground city",0.03,3,43.648429,-79.382280,Petit Four Bakery,43.647744,-79.379588,Sandwich Place


In [42]:
#cluster 4
restaurant_merge.loc[restaurant_merge['Cluster'] == 4]

,Neighborhood,Japanese Restaurant,Cluster,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Berczy Park,0.017241,4,43.644771,-79.373306,LCBO,43.642944,-79.372440,Liquor Store
29,St. James Town,0.012821,4,43.651494,-79.375418,Hotel Victoria,43.648084,-79.377582,Hotel
14,"Harbourfront East, Union Station, Toronto Islands",0.010000,4,43.640816,-79.381752,Radisson Admiral Hotel Toronto-Harbourfront,43.638847,-79.385584,Hotel
14,"Harbourfront East, Union Station, Toronto Islands",0.010000,4,43.640816,-79.381752,Panago,43.642518,-79.383591,Pizza Place
14,"Harbourfront East, Union Station, Toronto Islands",0.010000,4,43.640816,-79.381752,Sushi Shop,43.643485,-79.380960,Restaurant
...,...,...,...,...,...,...,...,...,...
25,"Richmond, Adelaide, King",0.010753,4,43.650571,-79.384568,Pi Co.,43.648651,-79.385874,Pizza Place
25,"Richmond, Adelaide, King",0.010753,4,43.650571,-79.384568,deKEFIR,43.650433,-79.379393,Ice Cream Shop
25,"Richmond, Adelaide, King",0.010753,4,43.650571,-79.384568,Equinox Bay Street,43.648100,-79.379989,Gym
25,"Richmond, Adelaide, King",0.010753,4,43.650571,-79.384568,Kojin,43.649398,-79.386091,Colombian Restaurant


In [43]:
#map
map_jap = folium.Map(location=[lat, lon], zoom_start=5)

#colors 
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
markers_colors = []

# add map
for lat, lon, poi, cluster in zip(restaurant_merge['Neighborhood Latitude'], restaurant_merge['Neighborhood Longitude'], 
                                  restaurant_merge['Neighborhood'], restaurant_merge['Cluster']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster))
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_jap)
       
map_jap